Import Libraries

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import os
import sys

Device Setup:
If you have an Nvidia GPU, you need to install CUDA
Otherwise, CPU will be used

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Hardware to be used:", device)
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(torch.cuda.get_device_name())

Hardware to be used: cuda:0
CUDA version: 12.1
NVIDIA GeForce RTX 2080 Ti


Enter the parent path (e.g., "C:\\Users\\{USERNAME}\\alzheimer-project-ai4all")

Enter path to dataset (*.npz)

Enter path to test dataset (e.g., "assets\Kaggle\alzheimer_mri_preprocessed_dataset\Test_Data")

In [14]:
# ENTER PARENT PATH - Example: C:\Users\{USERNAME}\ai-alzheimer-detection
parent_path = r"ENTER PARENT PATH"

dataset_dir = fr"{parent_path}\assets\Kaggle\alzheimer_mri_preprocessed_dataset"

train_dir = fr"Train_Data\augmented_adasyn_train_data.npz"
train_path = os.path.join(dataset_dir, train_dir)

val_dir = fr"Validation_Data\val_data.npz"
val_path = os.path.join(dataset_dir, val_dir)

test_dir = fr"Test_Data\test_data.npz"
test_path = os.path.join(dataset_dir, test_dir)

model_dir = r"models"
model_path = os.path.join(parent_path, model_dir)

if not os.path.exists(model_path):
    os.makedirs(model_path)

model_dir = r"models\best_ad_model.pth"
model_path = os.path.join(parent_path, model_dir)

sys.path.append(parent_path)

In [15]:
classes = ["Non_Demented", "Very_Mild_Demented", "Mild_Demented", "Moderate_Demented"]

Defining the hyper-parameters

In [16]:
epoches = 25 # implement early stopping
learning_rate = 0.001
batch_size = 8
early_stop = 5

Loading the datasets

In [17]:
from alzheimersdetection.AlzheimerDataset import AlzheimerDataset

train_dataset = AlzheimerDataset(train_path)
val_dataset = AlzheimerDataset(val_path)
test_dataset = AlzheimerDataset(test_path)

In [18]:
print("Training Dataset Size:")
print(len(train_dataset))

print("\nValidation Dataset Size:")
print(len(val_dataset))

print("\nTesting Dataset Size:")
print(len(test_dataset))

Training Dataset Size:
15113

Validation Dataset Size:
1280

Testing Dataset Size:
1280


In [19]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_iter = iter(train_loader)
val_iter = iter(val_loader)
test_iter = iter(test_loader)

print("Training Dataloader Size: ", len(train_loader))
X, y = next(train_iter)
print(X, y)

Training Dataloader Size:  1890
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,

Create our model using PyTorch's Convolutional Neural Network

In [20]:
from alzheimersdetection.AlzheimerModel import AlzheimerCNN

total_samples = len(train_dataset)
n_iters = len(train_loader) # math.ceil(total_samples/batch_size)
model = AlzheimerCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Total Samples: ", total_samples)
print("Total Iterations: ", n_iters)

Total Samples:  15113
Total Iterations:  1890


Training our model

In [21]:
best_val_loss = float('inf')
stop_ctr = 0
total_train_losses = []
total_val_losses = []
total_val_acc = []
all_pred = []
all_labels = []

for epoch in range(epoches):
    model.train()
    running_loss = 0.0
    
    # these are the images and labels in the batch
    for i, (images, labels) in enumerate(train_loader):
        labels = labels.type(torch.LongTensor)
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward() # Backpropogation
        optimizer.step() # Update the weights!!!

        running_loss += loss.item() * images.size(0)

    # Set the evaluation to test the validation
    model.eval()
    val_loss = 0.0
    val_correct = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.type(torch.LongTensor)
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss += criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, prediction = torch.max(outputs, 1)
            val_correct += torch.sum(prediction == labels.data)

            all_pred.extend(prediction.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    running_loss = running_loss / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_correct.double() / len(val_loader.dataset)
    
    total_train_losses.append(running_loss)
    total_val_losses.append(val_loss)
    total_val_acc.append(val_acc)

    print(f'epoch {epoch+1}/{epoches}, training loss: {running_loss:.4f}, validation loss: {val_loss:.4f}, val acc: {val_acc:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        stop_ctr = 0
        torch.save(model.state_dict(), model_path)
    else:
        stop_ctr += 1

    if stop_ctr >= early_stop:
        print("Early Stop!")
        print(f"Stopped at Epoch: {epoch}")
        break

epoch 1/25, training loss: 0.8577, validation loss: 62.8518, val acc: 0.6578
epoch 2/25, training loss: 0.5401, validation loss: 48.1772, val acc: 0.7227
epoch 3/25, training loss: 0.3908, validation loss: 39.9376, val acc: 0.7930
epoch 4/25, training loss: 0.2760, validation loss: 27.0296, val acc: 0.8766
epoch 5/25, training loss: 0.1841, validation loss: 34.9830, val acc: 0.8531
epoch 6/25, training loss: 0.1280, validation loss: 25.9238, val acc: 0.8922
epoch 7/25, training loss: 0.0884, validation loss: 34.7530, val acc: 0.8781
epoch 8/25, training loss: 0.0704, validation loss: 20.1211, val acc: 0.9336
epoch 9/25, training loss: 0.0523, validation loss: 20.1824, val acc: 0.9320
epoch 10/25, training loss: 0.0482, validation loss: 28.8059, val acc: 0.9227
epoch 11/25, training loss: 0.0331, validation loss: 23.1303, val acc: 0.9469
epoch 12/25, training loss: 0.0442, validation loss: 44.5674, val acc: 0.8914
epoch 13/25, training loss: 0.0315, validation loss: 26.6806, val acc: 0.

Verify our predictions using testing data (never seen before)

In [22]:
from alzheimersdetection import AlzheimerMetrics

AlzheimerMetrics.run_metrics(model, test_loader, device, classes)

Non_Demented(0) - Accuracy: 0.95, Precision: 0.964516129032258, Recall: 0.934375
Very_Mild_Demented(1) - Accuracy: 0.94765625, Precision: 0.913232104121475, Recall: 0.9397321428571429
Mild_Demented(2) - Accuracy: 0.9765625, Precision: 0.8983957219251337, Recall: 0.9385474860335196
Moderate_Demented(3) - Accuracy: 0.99921875, Precision: 1.0, Recall: 0.9230769230769231


------
<p style="text-align: center;"> Made with ❤️ </p>
<p style="text-align: center;"> Darwin Xue </p>